### Importing Required Libraries
This section imports all the necessary libraries and modules for building and training the neural network, as well as for data preprocessing and visualization.

In [166]:
import os
import torch
import torchvision
import torch.nn as nn
import torchvision.transforms as transforms
import numpy as np
from PIL import Image
import torch.nn.functional as F
import torch.optim as optim
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from torch.utils.data import Dataset, DataLoader

### Import Dataset From CSV File

In [167]:
df = pd.read_csv('./Churn_Modelling.csv')
df

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9996,15606229,Obijiaku,771,France,Male,39,5,0.00,2,1,0,96270.64,0
9996,9997,15569892,Johnstone,516,France,Male,35,10,57369.61,1,1,1,101699.77,0
9997,9998,15584532,Liu,709,France,Female,36,7,0.00,1,0,1,42085.58,1
9998,9999,15682355,Sabbatini,772,Germany,Male,42,3,75075.31,2,1,0,92888.52,1


### 1 - Data Preprocessing
- Remove irrelevant columns

In [168]:
df = df.drop(['RowNumber', 'CustomerId', 'Surname'], axis=1)
df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...
9995,771,France,Male,39,5,0.00,2,1,0,96270.64,0
9996,516,France,Male,35,10,57369.61,1,1,1,101699.77,0
9997,709,France,Female,36,7,0.00,1,0,1,42085.58,1
9998,772,Germany,Male,42,3,75075.31,2,1,0,92888.52,1


- Encode categorical variables (Geography and Gender) into numeric form using techniques like one-hot encoding or label encoding

In [169]:
# Initialize LabelEncoder
label_encoder = LabelEncoder()

# Apply Label Encoding to 'Geography' and 'Gender'
df['Geography_encoded'] = label_encoder.fit_transform(df['Geography'])
df['Gender_encoded'] = label_encoder.fit_transform(df['Gender'])

# Drop the original categorical columns (optional)
df = df.drop(['Geography', 'Gender'], axis=1)
df

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_encoded,Gender_encoded
0,619,42,2,0.00,1,1,1,101348.88,1,0,0
1,608,41,1,83807.86,1,0,1,112542.58,0,2,0
2,502,42,8,159660.80,3,1,0,113931.57,1,0,0
3,699,39,1,0.00,2,0,0,93826.63,0,0,0
4,850,43,2,125510.82,1,1,1,79084.10,0,2,0
...,...,...,...,...,...,...,...,...,...,...,...
9995,771,39,5,0.00,2,1,0,96270.64,0,0,1
9996,516,35,10,57369.61,1,1,1,101699.77,0,0,1
9997,709,36,7,0.00,1,0,1,42085.58,1,0,0
9998,772,42,3,75075.31,2,1,0,92888.52,1,1,1


- Normalize numerical features such as CreditScore, Age, Balance, etc., to bring
them into a similar scale.

In [170]:
# Initialize MinMaxScaler
scaler = MinMaxScaler()

# Apply Min-Max Scaling to numerical features
df[['CreditScore', 'Age', 'Balance']] = scaler.fit_transform(df[['CreditScore', 'Age', 'Balance']])

df

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_encoded,Gender_encoded
0,0.538,0.324324,2,0.000000,1,1,1,101348.88,1,0,0
1,0.516,0.310811,1,0.334031,1,0,1,112542.58,0,2,0
2,0.304,0.324324,8,0.636357,3,1,0,113931.57,1,0,0
3,0.698,0.283784,1,0.000000,2,0,0,93826.63,0,0,0
4,1.000,0.337838,2,0.500246,1,1,1,79084.10,0,2,0
...,...,...,...,...,...,...,...,...,...,...,...
9995,0.842,0.283784,5,0.000000,2,1,0,96270.64,0,0,1
9996,0.332,0.229730,10,0.228657,1,1,1,101699.77,0,0,1
9997,0.718,0.243243,7,0.000000,1,0,1,42085.58,1,0,0
9998,0.844,0.324324,3,0.299226,2,1,0,92888.52,1,1,1


- Split Data into Input and Output

In [171]:
input_data = df.drop(['Exited'], axis=1)
# input_data=input_data.drop(['Id'],axis=1)
output_data = df['Exited']

input_data

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_encoded,Gender_encoded
0,0.538,0.324324,2,0.000000,1,1,1,101348.88,0,0
1,0.516,0.310811,1,0.334031,1,0,1,112542.58,2,0
2,0.304,0.324324,8,0.636357,3,1,0,113931.57,0,0
3,0.698,0.283784,1,0.000000,2,0,0,93826.63,0,0
4,1.000,0.337838,2,0.500246,1,1,1,79084.10,2,0
...,...,...,...,...,...,...,...,...,...,...
9995,0.842,0.283784,5,0.000000,2,1,0,96270.64,0,1
9996,0.332,0.229730,10,0.228657,1,1,1,101699.77,0,1
9997,0.718,0.243243,7,0.000000,1,0,1,42085.58,0,0
9998,0.844,0.324324,3,0.299226,2,1,0,92888.52,1,1


In [172]:
# convert data to numpy arrays
input_data=input_data.values
output_data=output_data.values
input_data

array([[5.38000000e-01, 3.24324324e-01, 2.00000000e+00, ...,
        1.01348880e+05, 0.00000000e+00, 0.00000000e+00],
       [5.16000000e-01, 3.10810811e-01, 1.00000000e+00, ...,
        1.12542580e+05, 2.00000000e+00, 0.00000000e+00],
       [3.04000000e-01, 3.24324324e-01, 8.00000000e+00, ...,
        1.13931570e+05, 0.00000000e+00, 0.00000000e+00],
       ...,
       [7.18000000e-01, 2.43243243e-01, 7.00000000e+00, ...,
        4.20855800e+04, 0.00000000e+00, 0.00000000e+00],
       [8.44000000e-01, 3.24324324e-01, 3.00000000e+00, ...,
        9.28885200e+04, 1.00000000e+00, 1.00000000e+00],
       [8.84000000e-01, 1.35135135e-01, 4.00000000e+00, ...,
        3.81907800e+04, 0.00000000e+00, 0.00000000e+00]])

- Split the dataset into training, validation, and testing subsets.

In [173]:
X_train , X_test,Y_train ,Y_test =train_test_split(input_data,output_data,test_size=0.2,random_state=41)

### 2. Prepare PyTorch Dataset and DataLoader
- Convert the preprocessed data into PyTorch tensors.

In [174]:
# convert to tensor
X_train=torch.FloatTensor(X_train)
X_test=torch.FloatTensor(X_test)
Y_train=torch.FloatTensor(Y_train)
Y_test=torch.FloatTensor(Y_test)

- Create a custom Dataset class that returns feature-target pairs.


In [175]:
class CustomerDataset(Dataset):
    def __init__(self, features, target):
        # Convert features and target to PyTorch tensors
        self.features = features
        self.target = target

    def __len__(self):
        return len(self.features)

    def __getitem__(self, idx):
        sample = {
            'features': self.features[idx],
            'target': self.target[idx]
        }
        return sample

In [176]:
dataset = CustomerDataset(X_train, Y_train)
dataloader = DataLoader(dataset, batch_size=2, shuffle=True)

# for batch in dataloader:
#     print(batch['features'])
#     print(batch['target'])

### 3. Define the Neural Network Model

In [177]:
class CustomerChurnModel(nn.Module):
    def __init__(self, input_size, hidden_size1, hidden_size2, output_size):
        super(CustomerChurnModel, self).__init__()
        # Input layer to first hidden layer
        self.fc1 = nn.Linear(input_size, hidden_size1)
        self.relu1 = nn.ReLU()
        
        # First hidden layer to second hidden layer
        self.fc2 = nn.Linear(hidden_size1, hidden_size2)
        self.relu2 = nn.ReLU()
        
        # Second hidden layer to output layer
        self.fc3 = nn.Linear(hidden_size2, output_size)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        out = self.fc1(x)
        out = self.relu1(out)
        out = self.fc2(out)
        out = self.relu2(out)
        out = self.fc3(out)
        out = self.sigmoid(out)
        return out

In [178]:
# Define the dimensions
input_size = 10  # Number of input features
hidden_size1 = 16  # Neurons in the first hidden layer
hidden_size2 = 8   # Neurons in the second hidden layer
output_size = 1    # Single output neuron for binary classification

# Create an instance of the model
model = CustomerChurnModel(input_size, hidden_size1, hidden_size2, output_size)

### 4. Loss Function and Optimizer

In [179]:
# Define the loss function and optimizer
criterion = nn.BCELoss()  # Binary Cross-Entropy Loss
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)  # Adam optimizer
# optimizer = optim.SGD(model.parameters(), lr=0.01)  # SGD optimizer with learning rate 0.01

### 5. Train the Model

In [180]:
import torch
from torch.utils.data import DataLoader, TensorDataset

# Assuming X_train and Y_train are already defined as tensors
# Example:
# X_train = torch.randn(8000, 1)  # 8000 samples, 1 feature each
# Y_train = torch.randint(0, 2, (8000,))  # 8000 binary labels

# Step 1: Wrap your data into a TensorDataset
train_dataset = TensorDataset(X_train, Y_train)

# Step 2: Create a DataLoader
batch_size = 32  # Adjust batch size as needed
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

# Step 4: Training loop with DataLoader
num_epochs = 10
for epoch in range(num_epochs):
    for batch_idx, (inputs, targets) in enumerate(train_loader):
        # Forward pass
        outputs = model(inputs)  # Model predictions
        targets = targets.view(-1, 1).float()  # Ensure targets match output shape and type

        # Compute loss
        loss = criterion(outputs, targets)

        # Backward pass and optimization
        optimizer.zero_grad()  # Clear gradients
        loss.backward()        # Compute gradients
        optimizer.step()       # Update weights

        # Print loss for monitoring (optional: print every few batches)
        if (batch_idx + 1) % 10 == 0:  # Print every 10 batches
            print(f"Epoch [{epoch+1}/{num_epochs}], Batch [{batch_idx+1}/{len(train_loader)}], Loss: {loss.item():.4f}")

    # Print epoch-level loss (optional)
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}")

Epoch [1/10], Batch [10/250], Loss: 75.0000
Epoch [1/10], Batch [20/250], Loss: 84.3750
Epoch [1/10], Batch [30/250], Loss: 72.1302
Epoch [1/10], Batch [40/250], Loss: 81.2500
Epoch [1/10], Batch [50/250], Loss: 58.5711
Epoch [1/10], Batch [60/250], Loss: 11.7378


Epoch [1/10], Batch [70/250], Loss: 15.6250
Epoch [1/10], Batch [80/250], Loss: 31.2500
Epoch [1/10], Batch [90/250], Loss: 16.4273
Epoch [1/10], Batch [100/250], Loss: 34.3750
Epoch [1/10], Batch [110/250], Loss: 18.7500
Epoch [1/10], Batch [120/250], Loss: 27.7118
Epoch [1/10], Batch [130/250], Loss: 18.7500
Epoch [1/10], Batch [140/250], Loss: 11.0411
Epoch [1/10], Batch [150/250], Loss: 16.3606
Epoch [1/10], Batch [160/250], Loss: 18.7257
Epoch [1/10], Batch [170/250], Loss: 25.0027
Epoch [1/10], Batch [180/250], Loss: 26.3626
Epoch [1/10], Batch [190/250], Loss: 28.2861
Epoch [1/10], Batch [200/250], Loss: 7.8014
Epoch [1/10], Batch [210/250], Loss: 6.2502
Epoch [1/10], Batch [220/250], Loss: 16.3084
Epoch [1/10], Batch [230/250], Loss: 13.8468
Epoch [1/10], Batch [240/250], Loss: 28.1250
Epoch [1/10], Batch [250/250], Loss: 34.3750
Epoch [1/10], Loss: 34.3750
Epoch [2/10], Batch [10/250], Loss: 21.8750
Epoch [2/10], Batch [20/250], Loss: 25.0000
Epoch [2/10], Batch [30/250], Loss

### Testing

In [181]:
import torch
from torch.utils.data import DataLoader, TensorDataset
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Step 1: Prepare the test data
# Assuming X_test and Y_test are already defined as tensors
# Example:
# X_test = torch.randn(2000, 1)  # 2000 samples, 1 feature each
# Y_test = torch.randint(0, 2, (2000,))  # 2000 binary labels

# Wrap test data into a TensorDataset
test_dataset = TensorDataset(X_test, Y_test)

# Create a DataLoader for the test set
batch_size = 32  # Same as training batch size
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Step 2: Evaluate the model
model.eval()  # Set the model to evaluation mode
all_predictions = []
all_targets = []

with torch.no_grad():  # Disable gradient computation
    for inputs, targets in test_loader:
        outputs = model(inputs)  # Forward pass
        predictions = (outputs >= 0.5).float()  # Convert probabilities to binary predictions (0 or 1)
        
        # Collect predictions and targets
        all_predictions.extend(predictions.cpu().numpy())
        all_targets.extend(targets.cpu().numpy())

# Step 3: Calculate metrics
all_predictions = np.array(all_predictions)
all_targets = np.array(all_targets)

# Accuracy
accuracy = accuracy_score(all_targets, all_predictions)

# Precision
precision = precision_score(all_targets, all_predictions)

# Recall
recall = recall_score(all_targets, all_predictions)

# F1 Score
f1 = f1_score(all_targets, all_predictions)

# Print metrics
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")

Accuracy: 0.7990
Precision: 1.0000
Recall: 0.0025
F1 Score: 0.0050
